In [ ]:
!pip install arize-phoenix openai "arize-phoenix[evals]"

In [2]:
import phoenix as px
session = px.launch_app()

/Users/jeffreypeng/Projects/LLM_Project/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jeffreypeng/Projects/LLM_Project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:phoenix.config:📋 Ensuring phoenix working directory: /Users/jeffreypeng/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [26]:
from phoenix.trace.openai import OpenAIInstrumentor

# Initialize OpenAI auto-instrumentation
OpenAIInstrumentor().instrument()

import os
from openai import OpenAI

# Initialize an OpenAI client
client = OpenAI(api_key='')

# Define a conversation with a user message
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, can you help me with something?"}
]

# Generate a response from the assistant
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=conversation,
)

# Extract and print the assistant's reply
# The traces will be available in the Phoenix App for the above messsages
assistant_reply = response.choices[0].message.content

WARNI [opentelemetry.instrumentation.instrumentor] Attempting to instrument while already instrumented


In [7]:
# You can export a dataframe from the session
df = px.Client().get_spans_dataframe()

# Note that you can apply a filter if you would like to export only a sub-set of spans
# df = px.Client().get_spans_dataframe('span_kind == "RETRIEVER"')

/Users/jeffreypeng/Projects/LLM_Project/lib/python3.9/site-packages/phoenix/trace/dsl/query.py:746: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_attributes = pd.DataFrame.from_records(


In [9]:
df.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.openinference.span.kind,attributes.input.value,attributes.input.mime_type,attributes.llm.token_count.total,attributes.llm.model_name,attributes.llm.output_messages,attributes.output.mime_type,attributes.llm.token_count.completion,attributes.output.value,attributes.llm.token_count.prompt
context.span_id,,,,,,,,,,,,,,,,,,,,,
1459dd58d4cc48ec,ChatCompletion,LLM,None,2024-08-19 15:22:23.856855+00:00,2024-08-19 15:22:26.530612+00:00,ERROR,APIConnectionError: Connection error.,"[{'attributes': {'exception.escaped': 'False',...",1459dd58d4cc48ec,7bb35426b45941cd1567790bea2ba849,...,LLM,"{""messages"": [{""role"": ""system"", ""content"": ""Y...",application/json,NaN,None,None,None,NaN,None,NaN
9911ec6f836a86f3,ChatCompletion,LLM,None,2024-08-19 15:23:54.428521+00:00,2024-08-19 15:23:55.075062+00:00,OK,,[],9911ec6f836a86f3,bcc9ec85ca04aa9c3233667749e8d179,...,LLM,"{""messages"": [{""role"": ""system"", ""content"": ""Y...",application/json,45.0,gpt-3.5-turbo-0125,[{'message.content': 'Of course! I'll do my be...,application/json,19.0,"{""id"":""chatcmpl-9xyWYkCTNIOgRKYRlblRgo6ZSIJ4V""...",26.0


In [4]:
from urllib.request import urlopen
from phoenix.trace.trace_dataset import TraceDataset
from phoenix.trace.utils import json_lines_to_df

# Replace with the URL to your trace data
traces_url = "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/trace.jsonl"
with urlopen(traces_url) as response:
    lines = [line.decode("utf-8") for line in response.readlines()]
trace_ds = TraceDataset(json_lines_to_df(lines))


In [5]:
import phoenix as px

session = px.launch_app(trace=trace_ds)

# session = px.launch_app(trace=px.TraceDataset(trace_ds))

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [31]:
lines[0]

'{"name": "query", "context": {"trace_id": "f40dc5d5-08b7-4e23-80e1-2cd6e9f0cf29", "span_id": "bce5b9ae-4587-4ead-9ccc-de3fe29257bc"}, "span_kind": "CHAIN", "parent_id": null, "start_time": "2023-12-11T17:57:17.891021+00:00", "end_time": "2023-12-11T17:57:20.075141+00:00", "status_code": "OK", "status_message": "", "attributes": {"input.value": "How can I query for a monitor\'s status using GraphQL?", "input.mime_type": "text/plain", "output.value": "You can query for a monitor\'s status using GraphQL by including the \\"status\\" field in your query.", "output.mime_type": "text/plain"}, "events": [], "conversation": null}\n'

In [ ]:
from phoenix.evals import OpenAIModel, HallucinationEvaluator, QAEvaluator
from phoenix.evals import run_evals
import nest_asyncio
nest_asyncio.apply()  # This is needed for concurrency in notebook environments

# Set your OpenAI API key
api_key = ""  # Replace with your actual API key
eval_model = OpenAIModel(model="gpt-4-turbo-preview", api_key=api_key)

# Define your evaluators
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_evaluator = QAEvaluator(eval_model)

# Assume 'queries_df' is your input dataframe 
# for `hallucination_evaluator` your input df needs to have columns 'output', 'input', 'context'
# for `qa_evaluator` your input df needs to have columns 'output', 'input', 'reference'
assert all(column in queries_df.columns for column in ['output', 'input', 'context', 'reference'])

# Run the evaluators, each evaluator will return a dataframe with evaluation results
# We upload the evaluation results to Phoenix in the next step
hallucination_eval_df, qa_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_evaluator],
    provide_explanation=True
)

# Log the evaluations
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_eval_df)
)